<a href="https://colab.research.google.com/github/Shujaat123/Ear_Biometrics/blob/main/EarBiometric_DeepLSE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **EB-LSE: Ear Biometrics Using Deep Latent Space Encoding**


This code provide python implementation of EB-LSE algorithm.

# Loading Useful packages

In [93]:
## Load useful packages
!pip install wget
!pip install py7zr

import keras
import py7zr
from zipfile import ZipFile
from random import sample

import PIL.Image as Image

import matplotlib.pyplot as plt
from  sklearn.model_selection import train_test_split
from os import listdir
from os import path
import h5py
import keras.backend as K
import numpy as np
import tensorflow as tf
import wget
from keras import regularizers
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint
from keras.layers import *
from keras.models import Model
from keras.models import load_model
from keras.utils.np_utils import to_categorical
from sklearn.metrics import accuracy_score, balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report, accuracy_score, matthews_corrcoef, balanced_accuracy_score, precision_recall_fscore_support
from sklearn.metrics import auc, average_precision_score, precision_recall_curve, roc_curve


In [95]:
## Define performance measures
def yoden_index(y, y_pred):
  tn, fp, fn, tp = confusion_matrix(y, y_pred, labels=[0,1]).ravel()
  j = (tp/(tp+fn)) + (tn/(tn+fp)) - 1
  return j

def pmeasure(y, y_pred):
    tn, fp, fn, tp = confusion_matrix(y, y_pred, labels=[0,1]).ravel()
    sensitivity = tp / (tp + fn )
    specificity = tn / (tn + fp)
    f1score = (2 * tp) / (2 * tp + fp + fn)
    return ({'Sensitivity': sensitivity, 'Specificity': specificity, 'F1-Score': f1score})

def Show_Statistics(msg,Stats):
  print(msg.upper())
  print(70*'-')
  print('Accuracy:',Stats[0])
  print('Sensitivity:',Stats[1])
  print('Specificity:',Stats[2])
  print('F1-Score:',Stats[3])
  print('MCC:',Stats[4])
  print('Balance Accuracy:',Stats[5])
  print('Youden-Index:',Stats[6])
  print('AUC:',Stats[7])
  print('AUPR:',Stats[8])
  print('Reconstruction MSE:',Stats[9])
  print(70*'-')

def Calculate_Stats(y_actual,y_pred, y_score):
  acc = accuracy_score(y_actual.argmax(axis=1), y_pred.argmax(axis=1))
  sen = pmeasure(y_actual.argmax(axis=1), y_pred.argmax(axis=1))['Sensitivity']
  spe = pmeasure(y_actual.argmax(axis=1), y_pred.argmax(axis=1))['Specificity']
  f1 = pmeasure(y_actual.argmax(axis=1), y_pred.argmax(axis=1))['F1-Score']
  mcc = matthews_corrcoef(y_actual.argmax(axis=1), y_pred.argmax(axis=1))
  bacc = balanced_accuracy_score(y_actual.argmax(axis=1), y_pred.argmax(axis=1))
  yi = yoden_index(y_actual.argmax(axis=1), y_pred.argmax(axis=1))
  #auc = roc_auc_score(y_actual.argmax(axis=1), y_pred.argmax(axis=1))
  
  pre, rec, _ = precision_recall_curve(y_actual.argmax(axis=1), y_score, pos_label=1)
  fpr, tpr, _ = roc_curve(y_actual.argmax(axis=1), y_score, pos_label=1)
  auroc = auc(fpr, tpr)
  aupr = auc(rec, pre)
    
  return acc, sen, spe, f1, mcc, bacc, yi, auroc, aupr  

def label_by_th(y_pred, threshold=0.5):
  y_pred_copy = y_pred.copy()
  y_pred_copy[y_pred>= threshold] = 1 
  y_pred_copy[y_pred<threshold] = 0 
  return y_pred_copy

def cutoff_youdens_j(fpr,tpr,thresholds):
  j_scores = tpr-fpr
  j_ordered = sorted(zip(j_scores,thresholds))
  return j_ordered[-1][1]

# Pre-processing wheather-classification dataset

**Loading and processing dataset**

In [56]:
# data_path = 'https://github.com/Shujaat123/Ear_Biometrics/blob/main/datasets/AMI_dataset.zip?raw=true'
# filename = 'AMI_dataset.zip'
# if(path.exists(filename)):
#   !rm $filename
#   print('existing file:', filename, ' has been deleted')
# print('downloading latest version of file:', filename)
# wget.download(data_path, filename)
# print('DONE')

# with ZipFile(filename, 'r') as zipObj:
#    # Extract all the contents of zip file in current directory
#    zipObj.extractall()
# !ls

In [57]:
# # Processing AMI_dataset
# src_dir = 'AMI_dataset'
# images_name = listdir(src_dir)
# subjects = []
# for img_ind in range(0,len(images_name)):
#     subjects.append(int(images_name[img_ind].split('_')[0]))

# images_name_ord = []
# subjects_ord = []
# sub_ind = sorted(range(len(subjects)),key=subjects.__getitem__)
# for pos, item in enumerate(sub_ind):
#   images_name_ord.append(images_name[item])
#   subjects_ord.append(subjects[item])
# images_name = images_name_ord
# subjects = subjects_ord

# print(subjects)
# print(images_name)

# sub_unique = np.unique(subjects)
# img_ind = 0
# ear_images = []
# sub_labels = [];
# target_size = (123, 176)

# for sub_ind in range(0,len(sub_unique)):
#   for sample_ind in range(0,int(len(subjects)/len(sub_unique))):
#     img_path = src_dir+'/'+images_name[img_ind]
#     ear_img = (plt.imread(img_path))/255
    
#     ear_img = Image.open(img_path)
#     ear_img = ear_img.resize(target_size, Image.ANTIALIAS)
#     ear_img = np.asarray(ear_img).astype(np.float32)/255

#     ear_images.append(ear_img)
#     sub_labels.append(sub_ind)
#     img_ind = img_ind + 1

# ear_images = np.array(ear_images)
# sub_labels = to_categorical(np.array(sub_labels))

# print(ear_images.shape)
# print(sub_labels.shape)


In [58]:
data_path = 'https://github.com/Shujaat123/Ear_Biometrics/blob/main/datasets/IITD_Dataset.7z?raw=true'
filename = 'IITD_Dataset.7z'
if(path.exists(filename)):
  !rm $filename
  print('existing file:', filename, ' has been deleted')
print('downloading latest version of file:', filename)
wget.download(data_path, filename)
print('DONE')

with py7zr.SevenZipFile('IITD_Dataset.7z', mode='r') as z:
    z.extractall()
!ls

existing file: IITD_Dataset.7z  has been deleted
downloading latest version of file: IITD_Dataset.7z
DONE
 ear   IITD_Dataset.7z	'models\model-best.h5'	 sample_data


In [59]:
# Processing IITD_dataset
src_dir = 'ear/processed/221'
images_name = listdir(src_dir)
images_name_temp = []
subjects = []
for img_ind in range(0,len(images_name)):
  if(not(images_name[img_ind]=='Thumbs.db')):
    subjects.append(int(images_name[img_ind].split('_')[0]))
    images_name_temp.append(images_name[img_ind])

images_name = images_name_temp
images_name_ord = []
subjects_ord = []

sub_ind = sorted(range(len(subjects)),key=subjects.__getitem__)
for pos, item in enumerate(sub_ind):
  images_name_ord.append(images_name[item])
  subjects_ord.append(subjects[item])

images_name = images_name_ord
subjects = subjects_ord

print(subjects)
print(images_name)

img_ind = 0
ear_images = []
sub_labels = [];
target_size = (180, 180)

for sub_ind in range(0,len(subjects)):
  img_path = src_dir+'/'+images_name[sub_ind]
  ear_img = (plt.imread(img_path))/255
  
  ear_img = Image.open(img_path)
  ear_img = ear_img.resize(target_size, Image.ANTIALIAS)
  ear_img = np.asarray(ear_img).astype(np.float32)/255

  ear_images.append(ear_img)
  sub_labels.append(subjects[sub_ind]-1)

ear_images = np.array(ear_images)
sub_labels = to_categorical(np.array(sub_labels))

ear_images = np.expand_dims(ear_images, axis=3)
# sub_labels = np.expand_dims(sub_labels, axis=2)

ear_images = np.concatenate((ear_images,ear_images,ear_images), axis=3)

print(ear_images.shape)
print(sub_labels.shape)


[1, 1, 1, 1, 1, 1, 2, 2, 2, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4, 4, 5, 5, 5, 5, 6, 6, 6, 7, 7, 7, 7, 8, 8, 8, 9, 9, 9, 10, 10, 10, 11, 11, 11, 12, 12, 12, 13, 13, 13, 13, 13, 13, 14, 14, 14, 15, 15, 15, 16, 16, 16, 16, 17, 17, 17, 18, 18, 18, 19, 19, 19, 19, 20, 20, 20, 21, 21, 21, 22, 22, 22, 23, 23, 23, 23, 24, 24, 24, 24, 25, 25, 25, 25, 26, 26, 26, 27, 27, 27, 27, 27, 28, 28, 28, 29, 29, 29, 29, 30, 30, 30, 30, 31, 31, 31, 32, 32, 32, 32, 33, 33, 33, 34, 34, 34, 35, 35, 35, 35, 36, 36, 36, 37, 37, 37, 37, 38, 38, 38, 39, 39, 39, 40, 40, 40, 41, 41, 41, 41, 42, 42, 42, 42, 43, 43, 43, 43, 43, 44, 44, 44, 45, 45, 45, 46, 46, 46, 46, 47, 47, 47, 48, 48, 48, 49, 49, 49, 49, 49, 50, 50, 50, 50, 50, 51, 51, 51, 52, 52, 52, 52, 53, 53, 53, 54, 54, 54, 54, 54, 55, 55, 55, 55, 55, 56, 56, 56, 56, 57, 57, 57, 57, 58, 58, 58, 58, 58, 59, 59, 59, 59, 59, 60, 60, 60, 61, 61, 61, 62, 62, 62, 63, 63, 63, 63, 63, 63, 64, 64, 64, 64, 64, 64, 65, 65, 65, 66, 66, 66, 66, 67, 67, 67, 68, 68, 68, 68, 69, 69, 

In [60]:
# ear_images = np.concatenate((ear_images,ear_images,ear_images), axis=3)
# print(ear_images.shape)

**Generate Training, Validation and Test datasets**

In [61]:
X_train, X_test, y_train, y_test = train_test_split(ear_images, sub_labels, test_size=0.142857142857, random_state=42)
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

ear_images = None

(679, 180, 180, 3)
(679, 221)
(114, 180, 180, 3)
(114, 221)


## **From: A deep learning approach for person dentification using ear biometrics**

In [88]:
def BUS_Final_Model(num_filters=8,input_shape=(180, 50, 1)):
    # Encoder Network
    encoder_input = Input(shape=input_shape, name='encoder_input')
    enc_l1 = Conv2D(num_filters, 3, activation='tanh', name='encoder_layer1', padding='same')(encoder_input)
    enc_l1 = MaxPooling2D(pool_size=(2, 2))(enc_l1)
    enc_l1 = Conv2D(2*num_filters, 3, activation='tanh', name='encoder_layer2', padding='same')(enc_l1)
    enc_l1 = BatchNormalization()(enc_l1)
    enc_l1 = MaxPooling2D(pool_size=(2, 2))(enc_l1)
    # enc_l1 = Dropout(0.3)(enc_l1)
    

    enc_l2 = Conv2D(4*num_filters, 3, activation='tanh', name='encoder_layer3', padding='same')(enc_l1)
    enc_l2 = MaxPooling2D(pool_size=(2, 2))(enc_l2)
    enc_l2 = Conv2D(4*num_filters, 3, activation='tanh', name='encoder_layer4', padding='same',
                    kernel_regularizer=regularizers.l1_l2(l1=1e-5, l2=1e-4),
                    bias_regularizer=regularizers.l2(1e-4),
                    activity_regularizer=regularizers.l2(1e-5))(enc_l2)
    enc_l2 = MaxPooling2D(pool_size=(2, 2))(enc_l2)
    enc_l2 = Dropout(0.1)(enc_l2)
    

    enc_l3 = Conv2D(8*num_filters, 3, activation='tanh', name='encoder_layer5', padding='same')(enc_l2)
    enc_l3 = BatchNormalization()(enc_l3)
    enc_l3 = MaxPooling2D(pool_size=(2, 2))(enc_l3)
    enc_l3 = Conv2D(16*num_filters, 3, activation='tanh', name='encoder_layer6', padding='same')(enc_l3)
    enc_l3 = BatchNormalization()(enc_l3)
    # enc_l3 = Dropout(0.3)(enc_l3)

    # Classifier Network
    # flat = GlobalAveragePooling2D()(enc_l3)
    flat = Flatten()(enc_l3)
    class_output = Dense(221, activation='softmax', name='class_output')(flat)

    model = Model(inputs=[encoder_input], outputs=[class_output])  # class_output, decoder_output,

    # Compiling model
    model.compile(optimizer=tf.keras.optimizers.Adam(lr=1e-4),#RMSprop(),#learning_rate=0.0001, beta_1=0.9, beta_2=0.999,
                                                      #epsilon=1e-07, amsgrad=False),
                  loss= {'class_output': keras.losses.BinaryCrossentropy(from_logits=True)}, # ''categorical_crossentropy'}, #loss_DSSIM},
                  metrics={'class_output': 'accuracy'})#[DSSIM, DPSNR, 'mae']})
    model.summary()
    return model

In [63]:
# def BUS_Final_Model(num_filters=8,input_shape=(180, 50, 1)):
#   base_model = keras.applications.Xception(
#       weights='imagenet',  # Load weights pre-trained on ImageNet.
#       # weights=None,  # Load weights pre-trained on ImageNet.
#       input_shape=input_shape,
#       include_top=False)  # Do not include the ImageNet classifier at the top.
#   base_model.trainable = False
#   # base_model.trainable = True
#   # Encoder Network
#   encoder_input = Input(shape=input_shape, name='encoder_input')
#   # We make sure that the base_model is running in inference mode here,
#   # by passing `training=False`. This is important for fine-tuning, as you will
#   # learn in a few paragraphs.
#   x = base_model(encoder_input, training=True)
#   # Convert features of shape `base_model.output_shape[1:]` to vectors
#   x = keras.layers.GlobalAveragePooling2D()(x)
#   # A Dense classifier with a single unit (binary classification)
#   class_output = keras.layers.Dense(221, name='class_output')(x)
#   model = keras.Model(encoder_input, class_output)

#   model.compile(optimizer=keras.optimizers.Adam(lr=0.001),
#                 loss=keras.losses.BinaryCrossentropy(from_logits=True),
#                 metrics=['accuracy'])
#                 # metrics=[keras.metrics.BinaryAccuracy()])
#   model.summary()
#   return model

# Designing an Auto-Encoder-based classification model

In [89]:
model = BUS_Final_Model(num_filters=8,input_shape=(X_train.shape[1:]))

Model: "model_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
encoder_input (InputLayer)   [(None, 180, 180, 3)]     0         
_________________________________________________________________
encoder_layer1 (Conv2D)      (None, 180, 180, 8)       224       
_________________________________________________________________
max_pooling2d_35 (MaxPooling (None, 90, 90, 8)         0         
_________________________________________________________________
encoder_layer2 (Conv2D)      (None, 90, 90, 16)        1168      
_________________________________________________________________
batch_normalization_33 (Batc (None, 90, 90, 16)        64        
_________________________________________________________________
max_pooling2d_36 (MaxPooling (None, 45, 45, 16)        0         
_________________________________________________________________
encoder_layer3 (Conv2D)      (None, 45, 45, 32)        4640

# Model Training

In [90]:
es = EarlyStopping(monitor='val_loss', mode='min', verbose=0, patience=100)
checkpoint = ModelCheckpoint('models\\model-best.h5', verbose=1, monitor='val_loss',save_best_only=True, mode='auto')
  
history = model.fit({'encoder_input': X_train}, {'class_output': y_train}, validation_data=({'encoder_input': X_test}, {'class_output': y_test}),
                    batch_size=1, epochs=1000, shuffle=True, verbose=2, callbacks=[checkpoint, es])

Epoch 1/1000
679/679 - 3s - loss: 0.0622 - accuracy: 0.0309 - val_loss: 0.0768 - val_accuracy: 0.0000e+00

Epoch 00001: val_loss improved from inf to 0.07679, saving model to models\model-best.h5
Epoch 2/1000
679/679 - 2s - loss: 0.0320 - accuracy: 0.2592 - val_loss: 0.0708 - val_accuracy: 0.1316

Epoch 00002: val_loss improved from 0.07679 to 0.07080, saving model to models\model-best.h5
Epoch 3/1000
679/679 - 2s - loss: 0.0209 - accuracy: 0.7393 - val_loss: 0.0584 - val_accuracy: 0.3158

Epoch 00003: val_loss improved from 0.07080 to 0.05840, saving model to models\model-best.h5
Epoch 4/1000
679/679 - 2s - loss: 0.0142 - accuracy: 0.9529 - val_loss: 0.0623 - val_accuracy: 0.3860

Epoch 00004: val_loss did not improve from 0.05840
Epoch 5/1000
679/679 - 2s - loss: 0.0111 - accuracy: 1.0000 - val_loss: 0.0471 - val_accuracy: 0.5263

Epoch 00005: val_loss improved from 0.05840 to 0.04714, saving model to models\model-best.h5
Epoch 6/1000
679/679 - 2s - loss: 0.0095 - accuracy: 1.0000 - 

# Performance Evaluation

In [128]:
del model
model = load_model('models\model-best.h5')              
# y_train_pred = model.predict(X_train, batch_size=1, verbose=0)
y_test_pred = model.predict(X_test, batch_size=1, verbose=0)

# y_train_score = y_train_pred[:,1]
# y_test_score = y_test_pred[:,1]

# y_train_score = y_train_pred
# y_test_score = y_test_pred

In [129]:

# y_train_pred1 = to_categorical(y_train_pred.argmax(axis=1), num_classes=y_train_pred.shape[1])

# tr_acc, tr_sen, tr_spe, tr_f1, tr_mcc, tr_bacc, tr_yi, tr_auc, tr_aupr = Calculate_Stats(y_train,y_train_pred1, y_train_score);



In [134]:
# y_test_pred1 = to_categorical(y_test_pred.argmax(axis=1), num_classes=y_test_pred.shape[1])
# ts_acc, ts_sen, ts_spe, ts_f1, ts_mcc, ts_bacc, ts_yi, ts_auc, ts_aupr = Calculate_Stats(y_test,y_test_pred1, y_test_score);
acc = accuracy_score(y_test.argmax(axis=1), y_test_pred.argmax(axis=1))

acc



0.868421052631579